In [1]:
import os
from src.customModel.customModel import TorchAttentionModel3, TorchAttentionModel4SAC, TorchVectorQNet, TorchVectorPolicyNet

from src.constant import SRC_PATH
os.environ["L5KIT_DATA_FOLDER"] = '/workspace/datasets'
# os.environ['CUDA_VISIBLE_DEVICES']= '0'
# os.environ["TUNE_RESULT_DIR"] =  '/DATA/l5kit/rllib_tb_logs'
import gym
from l5kit.configs import load_config_data
from l5kit.environment.envs.l5_env import SimulationConfigGym, GymStepOutput, L5Env
from l5kit.environment.envs.l5_env2 import SimulationConfigGym, GymStepOutput, L5Env2
from l5kit.visualization.visualizer.zarr_utils import episode_out_to_visualizer_scene_gym_cle
from l5kit.visualization.visualizer.visualizer import visualize
from bokeh.io import output_notebook, show
from l5kit.environment.gym_metric_set import L2DisplacementYawMetricSet, CLEMetricSet
from prettytable import PrettyTable
import datetime
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
import torch.nn as nn
import numpy as np
import gym
from typing import Dict
import numpy as np
import ray
import pytz
from ray import tune

ray.init(num_cpus=64, ignore_reinit_error=True, log_to_driver=False, object_store_memory = 5*10**9, local_mode=False)


from l5kit.configs import load_config_data

# get environment config
# env_config_path = '/workspace/source/configs/gym_config_history3.yaml'
# env_config_path = '/workspace/source/configs/gym_config84.yaml'
env_config_path = SRC_PATH + 'src/configs/gym_vectorizer_config.yaml'
env_config_path = SRC_PATH + 'src/configs/gym_vectorizer_config_hist3.yaml'
# env_config_path = '/workspace/source/src/configs/gym_vectorizer_config.yaml'
cfg = load_config_data(env_config_path)


#################### Define Training and Evaluation Environments ####################
# n_channels = (cfg['model_params']['future_num_frames'] + 1)* 2 + 3
# print(cfg['model_params']['future_num_frames'], cfg['model_params']['history_num_frames'], n_channels)
from ray import tune
from src.customEnv.wrapper import L5EnvWrapper, L5EnvWrapperTorch
eval_eps_length = 32
eval_sim_cfg = SimulationConfigGym()
eval_sim_cfg.num_simulation_steps = eval_eps_length + 1

# Register , how your env should be constructed (always with 5, or you can take values from the `config` EnvContext object):
env_kwargs = {'env_config_path': env_config_path, 'use_kinematic': True, 'sim_cfg': eval_sim_cfg}

tune.register_env("L5-CLE-V2", lambda config: L5Env2(**env_kwargs))
ModelCatalog.register_custom_model( "TorchAttentionModel3", TorchAttentionModel3)
ModelCatalog.register_custom_model( "TorchAttentionModel4", TorchAttentionModel4SAC)
ModelCatalog.register_custom_model( "TorchVectorQNet", TorchVectorQNet)
ModelCatalog.register_custom_model( "TorchVectorPolicyNet", TorchVectorPolicyNet)



2023-04-25 08:41:09,286	WARNING services.py:1732 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 66764800 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=5.12gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2023-04-25 08:41:10,452	INFO worker.py:1538 -- Started a local Ray instance.


In [2]:
import ray
from ray import air, tune
hcmTz = pytz.timezone("Asia/Ho_Chi_Minh") 
date = datetime.datetime.now(hcmTz).strftime("%d-%m-%Y_%H-%M-%S")
ray_result_logdir = '/home/pronton/ray_results/debug_vector_sac_' + date

train_envs = 4
lr = 3e-4
lr_start = 3e-5
lr_end = 3e-6
lr = 3e-4
lr_start = 3e-5
lr_end = 3e-6
config_param_space = {
    "env": "L5-CLE-V2",
    "framework": "torch",
    "num_gpus": 1,
    "num_workers": 63,
    "num_envs_per_worker": train_envs,
    'q_model_config':{
        'custom_model': 'TorchVectorQNet',
        'custom_model_config': {'cfg': cfg,},
        "post_fcnet_hiddens": [256],
        "post_fcnet_activation": "relu",
    },
    'policy_model_config':{
        'custom_model': 'TorchVectorPolicyNet',
        'custom_model_config': {'cfg': cfg,},
        "post_fcnet_hiddens": [256],
        "post_fcnet_activation": "relu",
    },
    'tau': 0.005,
    'target_network_update_freq': 1,
    'replay_buffer_config':{
        'type': 'MultiAgentPrioritizedReplayBuffer',
        'capacity': int(1e6),
        "worker_side_prioritization": True,
    },
    'num_steps_sampled_before_learning_starts': 1024,
    
    'target_entropy': 'auto',
#     "model": {
#         "custom_model": "GN_CNN_torch_model",
#         "custom_model_config": {'feature_dim':128},
#     },
    '_disable_preprocessor_api': True,
     "eager_tracing": True,
     "restart_failed_sub_environments": True,
 
    # 'train_batch_size': 4000,
    # 'sgd_minibatch_size': 256,
    # 'num_sgd_iter': 16,
    # 'store_buffer_in_checkpoints' : False,
    'seed': 42,
    'batch_mode': 'truncate_episodes',
    "rollout_fragment_length": 1,
    'train_batch_size': 256, #512, #1024,#2048,
    'training_intensity' : 32, # (4x 'natural' value = 8) train_batch_size / (rollout_fragment_length x num_workers x num_envs_per_worker).
    'gamma': 0.8,
    'twin_q' : True,
    "lr": 3e-4,
    "min_sample_timesteps_per_iteration": 1024,
}

## Train

In [ ]:
result_grid = tune.Tuner(
    "SAC",
    run_config=air.RunConfig(
        stop={"episode_reward_mean": 0, 'timesteps_total': int(6e6)},
        local_dir=ray_result_logdir,
        checkpoint_config=air.CheckpointConfig(num_to_keep=2, 
                                               checkpoint_frequency = 10, 
                                               checkpoint_score_attribute = 'episode_reward_mean'),
        # callbacks=[WandbLoggerCallback(project="l5kit2", save_code = True, save_checkpoints = False),],
        ),
    param_space=config_param_space).fit()

## Viz

In [8]:
train_envs = 4
lr = 3e-4
lr_start = 3e-5
lr_end = 3e-6
lr = 3e-4
lr_start = 3e-5
lr_end = 3e-6
eval_config_param_space = {
    # "env": "L5-CLE-EVAL-V2",
    "framework": "torch",
    "num_gpus": 0,
    # "num_workers": 1,
    "num_envs_per_worker": train_envs,
    'q_model_config':{
        'custom_model': 'TorchVectorQNet',
        'custom_model_config': {'cfg': cfg,},
        "post_fcnet_hiddens": [256],
        "post_fcnet_activation": "relu",
    },
    'policy_model_config':{
        'custom_model': 'TorchVectorPolicyNet',
        'custom_model_config': {'cfg': cfg,},
        "post_fcnet_hiddens": [256],
        "post_fcnet_activation": "relu",
    },
    'tau': 0.005,
    'target_network_update_freq': 1,
    'replay_buffer_config':{
        'type': 'MultiAgentPrioritizedReplayBuffer',
        'capacity': int(1e6),
        "worker_side_prioritization": True,
    },
    'num_steps_sampled_before_learning_starts': 1024,
    
    'target_entropy': 'auto',
#     "model": {
#         "custom_model": "GN_CNN_torch_model",
#         "custom_model_config": {'feature_dim':128},
#     },
    '_disable_preprocessor_api': True,
     "eager_tracing": True,
     "restart_failed_sub_environments": True,
 
    # 'train_batch_size': 4000,
    # 'sgd_minibatch_size': 256,
    # 'num_sgd_iter': 16,
    # 'store_buffer_in_checkpoints' : False,
    'seed': 42,
    'batch_mode': 'truncate_episodes',
    "rollout_fragment_length": 1,
    'train_batch_size': 256, #512, #1024,#2048,
    'training_intensity' : 32, # (4x 'natural' value = 8) train_batch_size / (rollout_fragment_length x num_workers x num_envs_per_worker).
    'gamma': 0.8,
    'twin_q' : True,
    "lr": 3e-4,
    "min_sample_timesteps_per_iteration": 1024,
}

In [6]:
rollout_sim_cfg = SimulationConfigGym()
rollout_sim_cfg.num_simulation_steps = 50

# Register , how your env should be constructed (always with 5, or you can take values from the `config` EnvContext object):
env_kwargs = {'env_config_path': env_config_path, 
              'use_kinematic': True, 
              'sim_cfg': rollout_sim_cfg,  
              'train': False,
              'return_info': True}

rollout_env = L5Env2(**env_kwargs)
tune.register_env("L5-CLE-EVAL-V2", lambda config: L5Env2(**env_kwargs))

In [9]:
from ray.rllib.algorithms.sac import SAC
checkpoint_path = '/home/pronton/ray_results/vector_sac/SAC/SAC_L5-CLE-V2_a638f_00000_0_2023-04-24_14-52-35/checkpoint_000430'
model = SAC(config=eval_config_param_space, env='L5-CLE-EVAL-V2')
model.restore(checkpoint_path)

2023-04-25 08:42:55,834	WARNING deprecation.py:47 -- DeprecationWarning: `algo = Algorithm(env='L5-CLE-EVAL-V2', ...)` has been deprecated. Use `algo = AlgorithmConfig().environment('L5-CLE-EVAL-V2').build()` instead. This will raise an error in the future!
2023-04-25 08:43:00,670	WARNING catalog.py:637 -- Custom ModelV2 should accept all custom options as **kwargs, instead of expecting them in config['custom_model_config']!
2023-04-25 08:43:00,682	WARNING catalog.py:637 -- Custom ModelV2 should accept all custom options as **kwargs, instead of expecting them in config['custom_model_config']!
2023-04-25 08:43:00,695	WARNING catalog.py:637 -- Custom ModelV2 should accept all custom options as **kwargs, instead of expecting them in config['custom_model_config']!
2023-04-25 08:43:00,707	WARNING catalog.py:637 -- Custom ModelV2 should accept all custom options as **kwargs, instead of expecting them in config['custom_model_config']!
2023-04-25 08:43:00,720	WARNING catalog.py:637 -- Custom M

### Unroll scenes

In [6]:
from src.simulation.unrollGym import unroll, visualize_outputs
sim_outs = unroll(model, rollout_env, 3, cfg['gym_params']['max_val_scene_id'])

scene id: 0
scene id: 5406
scene id: 10812
scene id: 16218


In [10]:
rollout_env.set_reset_id(0)
obs = rollout_env.reset()
action = model.compute_single_action(obs, explore= False)
print(action)

# print(model.compute_single_action(obs2, deterministic = True))

[-0.0212633  -0.00472873 -0.11267066]


In [11]:
rollout_env.set_reset_id(99)
obs = rollout_env.reset()
action = model.compute_single_action(obs, explore= False)
print(action)

[-0.0212633  -0.00472873 -0.11267066]


In [91]:
rollout_env.set_reset_id(99)
obs2 = rollout_env.reset()
model.compute_action(obs2, explore = False)

/home/pronton/rlhf-car/l5kit2/l5kit/l5kit/data/zarr_dataset.py:213: RuntimeWarning: zarr dataset path should end with .zarr (for now). Open will fail for this dataset!
  dataset = ChunkedDataset("")
/home/pronton/rlhf-car/l5kit2/l5kit/l5kit/simulation/utils.py:107: RuntimeWarning: zarr dataset path should end with .zarr (for now). Open will fail for this dataset!
  new_dataset = ChunkedDataset("")


array([-0.0212633 , -0.00472873, -0.11267066], dtype=float32)

In [96]:
model.export_policy_model(export_dir='/home/pronton/sac_vector_model.pt')

In [99]:
import torch 
model = torch.load('/home/pronton/sac_vector_model/model.pt')

In [116]:
from torch.utils.data.dataloader import default_collate
obs_dict = default_collate(obs)
print(model(obs_dict))

KeyError: 0

In [48]:
model.compute_single_action(obs, deterministic = True)

array([-0.06268245, -0.01783037, -0.781215  ], dtype=float32)

In [49]:
def rollout_episode(model, env, idx = 0, num_simulation_steps = None):
        """Rollout a particular scene index and return the simulation output.

        :param model: the RL policy
        :param env: the gym environment
        :param idx: the scene index to be rolled out
        :return: the episode output of the rolled out scene
        """

        # Set the reset_scene_id to 'idx'
        env.set_reset_id(idx)

        # Rollout step-by-step
        obs = env.reset()
        done = False
        while True:
            # action, _ = model.predict(obs, deterministic=True)
            action = model.compute_action(obs)
            print('obs', obs)
            print('action', action)
            obs, _, done, info = env.step(action)
            if num_simulation_steps and idx >= num_simulation_steps:
                done = True
            if done:
                break

        # The episode outputs are present in the key "sim_outs"
        sim_out = info["sim_outs"][0]
        return sim_out
sim_outs = [rollout_episode(model, rollout_env, i) for i in range(1)]

/home/pronton/rlhf-car/l5kit2/l5kit/l5kit/data/zarr_dataset.py:213: RuntimeWarning: zarr dataset path should end with .zarr (for now). Open will fail for this dataset!
  dataset = ChunkedDataset("")
/home/pronton/rlhf-car/l5kit2/l5kit/l5kit/simulation/utils.py:107: RuntimeWarning: zarr dataset path should end with .zarr (for now). Open will fail for this dataset!
  new_dataset = ChunkedDataset("")


obs {'type': 3, 'all_other_agents_types': array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0]), 'agent_polyline_availability': array([ True,  True, False, False]), 'agent_trajectory_polyline': array([[ 4.5474735e-13,  0.0000000e+00,  0.0000000e+00],
       [-1.6674140e-01,  6.7835348e-04, -1.4495850e-04],
       [ 0.0000000e+00, -0.0000000e+00, -0.0000000e+00],
       [ 0.0000000e+00, -0.0000000e+00, -0.0000000e+00]], dtype=float32), 'other_agents_polyline': array([[[-9.60129452e+00,  3.51414025e-01,  3.84383202e-02],
        [-9.93279076e+00,  3.81967753e-01,  2.27804184e-02],
        [ 0.00000000e+00, -0.00000000e+00, -0.00000000e+00],
        [ 0.00000000e+00, -0.00000000e+00, -0.00000000e+00]],

       [[-2.37686520e+01,  1.00879803e+01, -1.59900403e+00],
        [-2.39033279e+01,  1.04113932e+01, -1.56962395e+00],
        [ 0.00000000e+00, -0.00000000e+00, -0.00000000e+00],
        [ 0.00000000e+00, -0.00000000e+00, -0.00000000e+

In [7]:
from src.validate.validator import compute_ade_fde, quantify_outputs
ades, fdes = compute_ade_fde(sim_outs)
quantify_outputs(sim_outs)

+----------+---------+--------+--------+------+------+-----+---------+
| scene_id |   FDE   |  ADE   |  DRT   |  CF  |  CR  |  CS |   PEGO  |
+----------+---------+--------+--------+------+------+-----+---------+
|    0     |  1.2886 | 0.7338 | 0.0683 | 0.0  | 0.0  | 0.0 | -0.2714 |
|   5406   |  3.336  | 1.1348 | 0.7187 | 0.0  | 11.0 | 0.0 |  0.6951 |
|  10812   |  3.336  | 1.1348 | 0.7187 | 0.0  | 7.0  | 0.0 |  0.6951 |
|  16218   | 18.3103 | 6.3129 | 3.1457 | 5.0  | 0.0  | 0.0 | -1.8709 |
| Overall  |  6.5677 | 2.3291 | 1.1629 | 1.25 | 4.5  | 0.0 |  -0.188 |
+----------+---------+--------+--------+------+------+-----+---------+


In [ ]:
output_notebook()
visualize_outputs(sim_outs)